# **Web scraping to indeed.com using the Python library Selenium.**

---

📎 README

This script executes code to scrape googletrends.com. It is necessary to input only the search terms and after save a CSV file with the name scraping_indeed.csv.

In [ ]:
# @title 1. ✨ Installing dependences for Selenium.

!pip3 install webdriver-manager &> /dev/null
!pip3 install selenium &> /dev/null
!pip install -U selenium &> /dev/null
!pip3 install pandas &> /dev/null
!pip3 install aiohttp &> /dev/null
!pip3 install asyncio &> /dev/null

import aiohttp
import asyncio
import threading
import pandas as pd
import selenium
import time

from time import sleep
from random import randint
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.common.keys import Keys
#from concurrent.futures import ThreadPoolExecutor, wait

In [ ]:
# @title 2. ✍ Run the script to scrape indeed.com.

job = "Data+Engineer" #@param {type:"string"}
location = "Washington" #@param {type:"string"}


# Finding location, position, radius=35 miles, sort by date and starting page
url = 'https://www.indeed.com/jobs?q={}&l={}&radius=35&filter=0&sort=date&start={}'

def web_driver():
    option= webdriver.ChromeOptions()
    option.add_argument("--incognito")
    # option.add_argument('--headless=chrome') # This is for to hidden windows
    # options.add_argument("--verbose")
    # options.add_argument('--no-sandbox')
    # options.add_argument('--headless')
    # options.add_argument('--disable-gpu')
    # options.add_argument("--window-size=1920, 1200")
    # options.add_argument('--disable-dev-shm-usage')
    # driver = webdriver.Chrome(options=options)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=option)
    return driver

job_lst=[]

def scraping_indeed(url,job,location):

    driver = web_driver()

    driver.get(url.format(job,location,0))

    sleep(randint(2, 6))

    # Pages for scraping
    pages=driver.find_element(By.CLASS_NAME,'jobsearch-JobCountAndSortPane-jobCount').text

    # Maxim iteractions for page.
    max_iter_pgs=int(pages.split(' ')[0])//15

    job_description_list_href=[]
    salary_list=[]

    for i in range(0,max_iter_pgs):
        driver.get(url.format(job_,location,i*10))

        sleep(randint(4, 6))

        job_page = driver.find_element(By.ID,"mosaic-jobResults")
        jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list

        for jj in jobs:
            job_title = jj.find_element(By.CLASS_NAME,"jcs-JobTitle")

            # Link of job.
            # ID job
            # Company name
            # Location
            # Posting date

            job_lst.append([job_title.text,
            job_title.get_attribute("href"),
            job_title.get_attribute("id"),
            jj.find_element(By.CLASS_NAME,"css-1x7z1ps").text,
            jj.find_element(By.CLASS_NAME,"css-t4u72d").text,
            jj.find_element(By.CSS_SELECTOR,"span.date").text])

            try:
                salary_list.append(jj.find_element(By.CLASS_NAME,"salary-snippet-container").text)

            except NoSuchElementException:
                try:
                    salary_list.append(jj.find_element(By.CLASS_NAME,"estimated-salary").text)

                except NoSuchElementException:
                    salary_list.append(None)

    driver.quit()

    return job_lst

start = time.time()
scraping = scraping_indeed(url,job,location)

names = ['Job', 'Link', 'ID job', 'Company', 'Location', 'Date']
scraping_df = pd.DataFrame(scraping, columns=names)
scraping_df['Date'] = scraping_df['Date'].str.replace('Posted\n', '')
scraping_df.to_csv('scraping_indeed.csv', index=False)

end = time.time()
print('Task completed in: ',end - start,' seconds')

scraping_df